In [ ]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten


from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re

In [ ]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
# dataset = dataset.loc[:,cols]
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class
dataset.sample(10)
dataset = shuffle(dataset)
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [ ]:
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class


In [ ]:
#X.dtype = 'int64'
# Y.dtype = 'int64'
# X = X.astype(int)

# X = X.astype(int)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
import os,sys
import shutil

shutil.rmtree('models')  
os.mkdir('models')

In [ ]:
# import tensorflow as tf
# def metric_precision(y_true,y_pred):
#     TP=tf.reduce_sum(y_true/tf.round(y_pred))
#     TN=tf.reduce_sum((1-y_true)(1-tf.round(y_pred)))
#     FP=tf.reduce_sum((1-y_true)tf.round(y_pred))
#     FN=tf.reduce_sum(y_true(1-tf.round(y_pred)))
#     precision=TP/(TP+FP)
#     return precision

In [ ]:
def accsorce(Y_test,yhat):
    sorce = 0
    for i in range(len(Y_test)):
        sorce += roc_auc_score(Y_test[i], yhat[i])

    return sorce/len(Y_test)

In [ ]:

#定义模型
import random
sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)

lstm_layers = [1,2,3,4,5]
dense_layers = [1,2,3]
units = [16,32,64,128]
dropout = [0.05,0.1,0.15,0.25]
Batch_size = [32,64,128]
optimizer = ['adam',sgd]
for the_batch_size in Batch_size:
    for the_dropout in dropout:
        for the_optimizer in optimizer:
            for the_dense_layers in dense_layers:
                for the_lstm_layers in lstm_layers:
                    for the_units in units:
                        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=random.randint(10,100))
                        X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
                        X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
                        model = Sequential()
                        # model.build(input_shape=(277,277,2))
                        #print(model.summary())
                        #model.add(SpatialDropout1D(0.2))
                        model.add(LSTM(the_units ,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences = True))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        # #第二层
                        for i in range(the_lstm_layers):
                            model.add(LSTM(the_units,return_sequences=True))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())

                        model.add(LSTM(the_units))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        #全连接层
                        for i in range(the_dense_layers):
                            model.add(Dense(the_dense_layers,activation='relu'))
                            model.add(Dropout(the_dropout))
                            
                        # model.add(Flatten()) 
                        
                        
                        model.add(Dense(2, activation='softmax'))
                        
                        #sgd = SGD(learning_rate=0.01, momentum=0.9 , decay=0.1, nesterov=False)
                        
                        # learning_rate = 0.1
                        # decay = 0.001
                        # epochs = 50
                        # batch_size = 64
                        
                        
                        model.compile(  loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                                        optimizer=the_optimizer, metrics=['categorical_accuracy'])
                        print(model.summary())

                        epochs = 100
                        batch_size = the_batch_size
                        filepath = './models/{categorical_accuracy:.4f}_{epoch:02d}_'+f'dropout_{the_dropout}pbatch_size_{the_batch_size}optimizer_{the_optimizer}dense_layers_{the_dense_layers}_lstm_layers_{the_lstm_layers}_unit_{the_units}.h5'
                        checkpoint = ModelCheckpoint(
                                            filepath=filepath,
                                            save_weights_only=False,
                                            monitor='categorical_accuracy',
                                            mode='max',
                                            save_best_only=True)
                        history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1,
                                            #callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
                                            callbacks=checkpoint)

In [ ]:
#model.save('mymodel.h5')

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/0.9373_68_dropout_0.05pbatch_size_32optimizer_adamdense_layers_3_lstm_layers_2_unit_64.h5')

In [ ]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
# dataset = dataset.loc[:,cols]
dataset.sample(10)
dataset = shuffle(dataset)
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=random.randint(10,100))
# X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [ ]:
#进行预测 make a prediction
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=random.randint(10,100))
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
# print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
yhat = best_model.predict(X_test)
# print(yhat.shape)
print(yhat)
print(Y_test)

In [ ]:
print(Y_test.shape)
print(yhat.shape)
# Y_test = Y_test.reshape(430)
# yhat = yhat.reshape(430)
# Y_test = list(Y_test)
# yhat = list(yhat)

In [ ]:
# print(yhat)

In [ ]:




sorce = 0
for i in range(len(Y_test)):
    sorce += roc_auc_score(Y_test[i], yhat[i])

print('ACC:',sorce/len(Y_test))
